In [ ]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

In [ ]:
import matplotlib
% matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import matplotlib.pyplot as plt


In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import time
import sys

In [ ]:
mv_listDF=pd.read_csv('mv_chartListDF_2016_1980.csv')
mv_listDF.columns

In [ ]:
DF=mv_listDF.drop(['Unnamed: 0'],axis=1)
DF

In [ ]:
# DF.shape[0], 
# len(DF)/1000 #14.258

In [ ]:
# http://www.boxofficemojo.com/movies/?id=pixar2015.htm


# Ghostbusters (2016)
# wikiurl= 'https://en.wikipedia.org/wiki/Ghostbusters_(2016_film)'
# wikiurl='https://en.wikipedia.org/wiki/Star_Wars:_The_Force_Awakens'

In [ ]:
# AllerrorList=[] 
# del AllerrorList

In [ ]:
mv_info=[]
errorList=[]

headers = ['mID', 'mName', 'mYrank',
           'mStudio', 'totalDGross',
           'totalDTheatre', 'openDGross', 'openDTheatre', 'openWkdate','closeWkdate',
           'F_gross','W_gross', 'PD_budget', 'Genre', 
           'mv_runtime', 'DURrelease', 
           'mv_Dir',
           #'mv_Write',#'mv_Act', 
           'mv_Prod', 'mv_Comp',
           'OscarNom','OscarInfoURL']

bomjURL = 'http://www.boxofficemojo.com'
mvIDurl = '/movies/?id='
endURL = '.htm'

   
# for m in range(0,2): #len(DF)#range(0,len(DF)):     
for m in range(0,len(DF)): #len(DF)#range(0,len(DF)): 
    try:
        mvID = DF.mID[m]
        mvName = DF.mName[m]
        mjurl = bomjURL+mvIDurl+mvID+endURL
        print(m)
        #print(m, mjurl)

        pgResp = requests.get(mjurl)
        # pgResp = requests.get(url)
        # pgResp = requests.get('http://www.boxofficemojo.com/movies/?id=pixar2015.htm')
        # pgResp = requests.get('http://www.boxofficemojo.com/movies/?id=piecesofapril.htm')
        # pgResp = requests.get('http://www.boxofficemojo.com/movies/?id=starwars7.htm') #(mjurl) #('http://www.boxofficemojo.com/movies/?id=pixar2015.htm')


        if pgResp.status_code == 200: 
            soup = BeautifulSoup(pgResp.text,'lxml')


            ForgTMP = soup.find(text = re.compile('Foreign:'))#.next.nextSibling.text.replace('\xa0','')
            if ForgTMP !=None: 
                ForGrossTMP = ForgTMP.next.nextSibling.text.replace('\xa0','')
                if ForGrossTMP.upper() != 'N/A':
                    FGross = int(ForGrossTMP.replace('\xa0','').replace('$','').replace(',',''))
                else:
                    FGross = np.nan
            else:
                FGross = np.nan

            WGrossTMP = soup.find(text = re.compile('Worldwide:'))#.next.nextSibling.text.replace('\xa0','')
            if WGrossTMP !=None: 
                WorldGrossTMP = WGrossTMP.next.nextSibling.text.replace('\xa0','')
                if WorldGrossTMP.upper() != 'N/A':
                    WGross = int(WorldGrossTMP.replace('\xa0','').replace('$','').replace(',',''))
                else:
                    WGross = np.nan
            else:
                WGross = np.nan
            WGross


            pdtBudgetTMP = soup.find(text = re.compile('Production Budget'))
            if pdtBudgetTMP !=None:
                pdtBtmp = pdtBudgetTMP.nextSibling.text
                if pdtBtmp != 'N/A':
                    if pdtBtmp.find(' million')>=0:
                        pdtBudget = int(pdtBtmp.split(' million')[0].replace('$','').replace('.','')+'0'*6)
                    else:
                        pdtBudget = int(pdtBtmp.replace('$','').replace(',',''))
                else:
                    pdtBudget = np.nan
            else:
                pdtBudget = np.nan


            genreTMP = soup.find(text = re.compile('Genre: '))
            if genreTMP != None:
                genreTmp0 = genreTMP.nextSibling.text
                if genreTmp0 != 'N/A':
                    genre = genreTmp0
                else:
                    genre = np.nan
            else:
                genre = np.nan

            runtimeTMP0 = soup.find(text = re.compile('Runtime: '))
            if runtimeTMP0 != None:
                runtimeTMP = runtimeTMP0.nextSibling.text
                if runtimeTMP != 'N/A':
                    H, tmp1,Min, tmp2 = runtimeTMP.split()
                    runtime = (int(H)*60)+int(Min)
                else:
                    runtime = np.nan
            else:
                runtime = np.nan

            RDurTMP = soup.find(text = re.compile('In Release:')) #||NoneType 
            if RDurTMP !=None:
                ReleaseDays = int(RDurTMP.next.next.text.split()[0])
            else:
                ReleaseDays = np.nan

            dirTMP = soup.find(text = re.compile('Director'))
            if dirTMP != None:
                director = dirTMP.next.text.split(' (co-director)')
            else:
                director = np.nan

            #         writeTMP = soup.find(text = re.compile('Writer'))
            #         if writeTMP !=None:
            #             wtmp000 = writeTMP.next.text.replace('J.J.','Jj').replace('McC','Mcc').replace('D\'S','Ds')
            #             wtmp00 = wtmp000.replace('C. Strouse','Strouse').replace('D. Brown','Brown').replace('Jean-Luc','Jeanluc')
            #             wtmp01 = wtmp00.replace('DuVall','Duvall').replace('Tim Blake','Timblake').replace('LaFauve','Lafauve')
            #             wtmp0 = wtmp01.replace('McQ','Mcq')
            #             writeTmp0 = re.findall('[A-Z][^A-Z]*', wtmp0)
            #             writer = [ writeTmp0[2*(n-1)]+writeTmp0[2*(n-1)+1] for n in range(1,round(len(writeTmp0)/2)+1) ]
            #         else:
            #             writer = np.nan


            #         actorsLtmp = soup.find(text = re.compile('Actor'))
            #         if actorsLtmp.next.text.find(' (Voice)')>0:
            #             actors = [n.split('*')[0] for n in actorsLtmp.next.text.replace(' ','').split('(Voice)')[:-1] ]
            #         #print(actorsNs)
            #         else:
            #             acttmp00 = actorsLtmp.next.text.replace(' ','').replace('TJ','Tj').replace('*(Cameo)','').replace('Common','')
            #             acttmp0 = acttmp00.replace('Smit-McPhee','Smitmcphee').replace('McC','Mcc').replace('K.W','Kw')
            #             acttmp = acttmp0.replace('L.J','Lj').replace('n-W','nw').replace('O\'C','Oc').replace('McG','Mcg')
            #             acttmp = acttmp.replace('MaryElizabethWinstead','MaryelizabethWinstead').replace('JackieEarle','Jackieearle')
            #             actors0 = re.findall('[A-Z][^A-Z]*', acttmp)
            #             searchJR = np.array([a.find('Jr.')+1 for a in actors0])
            #             if any(searchJR)!= False:
            #                 # searchJR.nonzero()
            #                 id = [i for i, x in enumerate(searchJR) if x][0]
            #                 actors0[id-1] = actors0[id-1]+'_Jr.'
            #                 actors0.pop(id)
            #                 #actors0
            #             else:
            #                 pass
            #             actors = [ actors0[2*(n-1)]+actors0[2*(n-1)+1] for n in range(1,round(len(actors0)/2)+1) ]
            #             #print(actorsNs)


            producerTMP = soup.find(text = re.compile('Producer'))
            if producerTMP !=None:
                prodTmp1 = producerTMP.next.find_all('a')
                producer = [p.text.replace('(executive)','') for p in prodTmp1]
                #proTmp0 = re.findall('[A-Z][^A-Z]*',producerTMP.next.text.replace('J.J.','Jj'))
                #producer = [ proTmp0[2*(n-1)]+proTmp0[2*(n-1)+1] for n in range(1,round(len(proTmp0)/2)+1) ]
            else:
                producer = np.nan


            compTMP = soup.find(text = re.compile('Composer'))#.next.text #|| NoneType
            if compTMP !=None:
                compTmp1 = compTMP.next.find_all('a')
                composer = [p.text.replace('(executive)','') for p in compTmp1]
                #comptmp = re.findall('[A-Z][^A-Z]*', compTMP.next.text )
                #composer = [ comptmp[2*(n-1)]+comptmp[2*(n-1)+1] for n in range(1,round(len(comptmp)/2)+1) ]
            else:
                composer = np.nan


            OawardTMP = soup.find(text = re.compile('Academy'))#.next.next.find('a')['href']  #|| NoneType
            if OawardTMP != None:
                OawardNom = 1
                OinfoURL = OawardTMP.next.next.find('a')['href']
            else:
                OawardNom = 0
                OinfoURL = np.nan


            mvInfo_dict = dict(zip(headers, [mvID, mvName, DF.mYrank[m],
                                             DF.mStudio[m], DF.totalDGross[m],
                                             DF.totalDTheatre[m], DF.openDGross[m], DF.openDTheatre[m],
                                             DF.openWkdate[m], DF.closeWkdate[m],
                                             FGross, WGross, pdtBudget, genre,
                                             runtime, ReleaseDays,
                                             director, 
                                             #writer,#actors, 
                                             producer, composer,
                                             OawardNom, OinfoURL]))
            mv_info.append(mvInfo_dict)

        time.sleep(0.2)
        
    except:
        e = sys.exc_info()[0]
        errorList.append((m, mjurl,e))
        print(m, mjurl,e)
        #write_to_page( "<p>Error: %s</p>" % e )

    
    
errorListDF = pd.DataFrame(errorList, columns =['mvID','mvPgURL', 'ErrType'])
errorListDF.to_csv('mvInfo_1-14258_ERRORS.csv')

mv_infoDF = pd.DataFrame(mv_info, columns=headers)
mv_infoDF.to_csv('mvInfo_1-14258_excludedError.csv')
    

In [ ]:
mv_info[-1]

In [ ]:
writeTmp0


In [ ]:
len(mv_info) , len(errorList)

In [ ]:
# AllerrorList.append(errorList)
# AllerrorList

In [ ]:
# errorList[0]
errorListDF = pd.DataFrame(errorList, columns =['mvID','mvPgURL', 'ErrType'])

In [ ]:
# errorListDF.to_csv('mvInfo_1-1000_ERRORs.csv')
errorListDF.to_csv('mvInfo_1-14258.csv')

In [ ]:
errorListDF

In [ ]:
mv_infoDF = pd.DataFrame(mv_info, columns=headers)

In [ ]:
mv_infoDF.iloc[528,:]

In [ ]:
# mv_infoDF.to_csv('mvInfo_1-1000_excludeError.csv')
mv_infoDF.to_csv('mvInfo_1-14258_excludedError.csv')
# 

In [ ]:
# mv_infoDF
# mv_infoDF.iloc[500,:]

In [ ]:
mv_info[1]

In [ ]:
# headers = ['mID', 'mName',
#            'mStudio', 'totalDGross',
#            'totalDTheatre', 'openDGross', 'openDTheatre', 'openWkdate','closeWkdate',
#            'F_gross','W_gross', 'PD_budget', 'Genre', 
#            'mv_runtime', 'DURrelease', 
#            'mv_Dir','mv_Write',
#            #'mv_Act', 
#            'mv_Prod', 'mv_Comp',
#            'OscarNom','OscarInfoURL']

# dict(zip(headers, [mvID, mvName,
#                    DF.mStudio[m], DF.totalDGross[m],
#                    DF.totalDTheatre[m], DF.openDGross[m], DF.openDTheatre[m],
#                    DF.openWkdate[m], DF.closeWkdate[m], 
#                    FGross, WGross, pdtBudget, genre,
#                    runtime, ReleaseDays,
#                    director, writer,
#                    #actors, 
#                    producer, composer,
#                    OawardNom, OinfoURL]))

In [ ]:
# pdtBtmp

In [ ]:
writeTmp0

In [ ]:
# pdtBudgetTMP = soup.find(text = re.compile('Production Budget'))
# if pdtBudgetTMP !=None:
#     pdtBtmp = pdtBudgetTMP.nextSibling.text
#     if pdtBtmp != 'N/A':
#         if pdtBtmp.find(' million')>=0:
#             pdtBudget = int(pdtBtmp.split(' million')[0].replace('$','').replace('.','')+'0'*6)
#         else:
#             pdtBudget = int(pdtBtmp.replace('$','').replace(',',''))
#     else:
#         pdtBudget = np.nan
# else:
#     pdtBudget = np.nan

# pdtBudget

In [ ]:
# genreTMP = soup.find(text = re.compile('Genre: '))
# if genreTMP != None:
#     genreTmp0 = genreTMP.nextSibling.text
#     if genreTmp0 != 'N/A':
#         genre = genreTmp0
#     else:
#         genre = np.nan
# else:
#     genre = np.nan
# genre

In [ ]:
writeTMP.next.text

In [ ]:
runtimeTMP0 = soup.find(text = re.compile('Runtime: '))
if runtimeTMP0 != None:
    runtimeTMP = runtimeTMP0.nextSibling.text
    if runtimeTMP != 'N/A':
        H, tmp1,Min, tmp2 = runtimeTMP.split()
        runtime = (int(H)*60)+int(Min)
    else:
        runtime = np.nan
else:
    runtime = np.nan
runtime